In [1]:
pip install transformers pandas scikit-learn

In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import pipeline

In [4]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
    

In [18]:
from datasets import load_dataset
dataset = load_dataset("imdb")

In [19]:
df = pd.DataFrame(dataset['train'])

In [20]:
df = df[0:500]

In [21]:
import re

def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

df['text'] = df['text'].apply(clean_text)

In [22]:
sentiment_analysis = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [23]:
df

,text,label
0,I rented I AM CURIOUSYELLOW from my video stor...,0
1,I Am Curious Yellow is a risible and pretentio...,0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godards Mas...,0
4,Oh brotherafter hearing about this ridiculous ...,0
...,...,...
495,I rarely make these comments but I felt compel...,0
496,This picture started out with good intentions ...,0
497,Cmon guys some previous reviewers have nearly ...,0
498,Okay now what the hell is this supposed to be ...,0


In [24]:
# Truncate text to fit within the model's max token limit
MAX_TOKENS = 512
def truncate_text(text, max_tokens=MAX_TOKENS):
    return text[:max_tokens]

df['text'] = df['text'].apply(truncate_text)

In [26]:
# Apply the sentiment analysis in batches
BATCH_SIZE = 32
def batch_sentiment_analysis(texts, batch_size=BATCH_SIZE):
    results = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        results.extend(sentiment_analysis(batch))
    return results

In [27]:
df['predicted_label'] = batch_sentiment_analysis(df['text'].tolist())

In [28]:
# Convert labels to binary format (POSITIVE/NEGATIVE)
df['predicted_label'] = df['predicted_label'].apply(lambda x: 1 if x['label'] == 'POSITIVE' else 0)
df['true_label'] = df['label'].apply(lambda x: 1 if x == 1 else 0)

In [29]:
# Split the data into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [30]:
# Evaluate the model's performance
accuracy = accuracy_score(test_df['true_label'], test_df['predicted_label'])
precision = precision_score(test_df['true_label'], test_df['predicted_label'])
recall = recall_score(test_df['true_label'], test_df['predicted_label'])
f1 = f1_score(test_df['true_label'], test_df['predicted_label'])

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

Accuracy: 0.89
Precision: 0.0
Recall: 0.0
F1-score: 0.0
